# <h1><center>Portfolio Selection Model</center></h1>
#### <h1><center>Model Creation</center></h1>

In [25]:
#import packages and intialize gurobi model file
import numpy as np
import mysql.connector
np.set_printoptions(suppress=True)
from gurobipy import *
import pandas as pd
m = Model('test')

# <h1><center>MySQL Connection</center></h1>


In [26]:
db = mysql.connector.connect(user = 'root',
                            password = 'root',
                            host = 'localhost',
                            database = 'nasdaq',
                            auth_plugin = 'mysql_native_password')

In [27]:
cur = db.cursor()
cur.execute('select stock1,stock2,covarience from cov')
q = cur.fetchall()
cur.execute('select stock,meanReturn from r')
r = cur.fetchall()

In [28]:
Q = {}
for i in range(len(q)):
    Q[(q[i][0],q[i][1])] = q[i][2]
    
R = {}
for i in range(len(r)):
    R[(r[i][0])] = r[i][1]
    
numStocks = len(R)

# <h1><center>Decision Variables</center></h1>


In [29]:
X = []
XNames = list(R.keys())
for i in range(len(XNames)):
    X.append(m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, name = XNames[i]))
m.update()

# <h1><center>Model Constraints</center></h1>


In [30]:
m.remove(m.getQConstrs())
m.addQConstr(quicksum(X[i] * Q[(XNames[i],XNames[j])] * X[j] for i in range(numStocks) for j in range(numStocks)), GRB.LESS_EQUAL, .65, "QCCC")
m.addConstr(quicksum( X[i] for i in range(len(X)) ), GRB.EQUAL,  1.00)
m.update()

# <h1><center>Objective Function</center></h1>


In [31]:
m.setObjective(quicksum( X[i]*R[XNames[i]] for i in range( len(R))),
               GRB.MAXIMIZE)
m.update()

# <h1><center>Results and Commands to Populate MySQL</center></h1>


In [32]:
m.optimize()

Optimize a model with 1 rows, 1158 columns and 1158 nonzeros
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [8e-09, 4e-01]
  Objective range  [8e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [7e-01, 7e-01]
Presolve time: 0.07s
Presolved: 121 rows, 1278 columns, 132059 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.259e+03
 Factor NZ  : 7.381e+03 (roughly 1 MByte of memory)
 Factor Ops : 5.979e+05 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.19313589e+00 -0.00000000e+00  1.16e+02 1.75e-01  1.51e-02     0s
   1   1.12953771e-02  1.67890347e-01  1.27e-04 5.55e-03  1.30e-04     0s
   2   1.41876602e-02  8.36660734e-02  1.40e-10 2.37e-03  5.77e-05     0s
   3   4.6709

In [33]:
m.setParam('OutputFlag', False) #Hide Gurobi output
portfolio = [] #[expReturn, risk] list

#For loop to calculate risk and returns 
max_risk = [i/1000 for i in range(0,1000,25)]
for k in max_risk:
    m.remove(m.getQConstrs())
    m.update()

    m.addQConstr(quicksum(X[i] * Q[(XNames[i],XNames[j])] * X[j] for i in range(numStocks) for j in range(numStocks)), GRB.LESS_EQUAL, k)
    m.update()

    m.optimize()
    if m.status == 2:
        portfolio.append([m.objVal, k])

portfolio

[[0.05465889541224628, 0.025],
 [0.05922762903660833, 0.05],
 [0.06209742571906666, 0.075],
 [0.06430495863158901, 0.1],
 [0.06616781820623249, 0.125],
 [0.0678097878871155, 0.15],
 [0.06929453166585595, 0.175],
 [0.0706254271621665, 0.2],
 [0.07180725802957359, 0.225],
 [0.072881519073433, 0.25],
 [0.07387310550480855, 0.275],
 [0.07479850491580253, 0.3],
 [0.07566937361561055, 0.325],
 [0.07649451786751857, 0.35],
 [0.07727945804989138, 0.375],
 [0.0780238111557997, 0.4],
 [0.0787312693173416, 0.425],
 [0.07935573532428951, 0.45],
 [0.07935572734592736, 0.475],
 [0.07935573832613767, 0.5],
 [0.07935573985263865, 0.525],
 [0.07935574031942137, 0.55],
 [0.07935574069028341, 0.575],
 [0.07935551571360544, 0.6],
 [0.07935549704986455, 0.625],
 [0.07935551758960067, 0.65],
 [0.07935558487326341, 0.675],
 [0.07935574067513297, 0.7],
 [0.079355740678662, 0.725],
 [0.07935574062392786, 0.75],
 [0.07935574051979102, 0.775],
 [0.07935574040959957, 0.8],
 [0.0793557380580757, 0.825],
 [0.079355

In [36]:
# Insert reutrn information
for i in range(len(portfolio)):    
    cur.execute("INSERT INTO portfolio(expReturn, expRisk) VALUES ({:f},{:f})".format(portfolio[i][0], portfolio[i][1]))

# Make sure data is committed to the database
db.commit()

cur.close()
db.close()

ProgrammingError: Cursor is not connected